In [17]:
!pip install -U --index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/Triton-Nightly/pypi/simple/ triton-nightly
!pip install -U --no-deps 'jax-triton[cuda11_pip] @ git+https://github.com/jax-ml/jax-triton.git'
#!pip install -Uq "jax[cuda11_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Looking in indexes: https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/Triton-Nightly/pypi/simple/
  Cloning https://github.com/jax-ml/jax-triton.git to /var/tmp/pip-install-1c4g3dee/jax-triton_2e00831c3b0d482a89bbef5d0233b1f0
  Running command git clone --filter=blob:none --quiet https://github.com/jax-ml/jax-triton.git /var/tmp/pip-install-1c4g3dee/jax-triton_2e00831c3b0d482a89bbef5d0233b1f0
  Resolved https://github.com/jax-ml/jax-triton.git to commit 15a479e01f89077bb53b9af6ec84355c97d16648
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for jax-triton: filename=jax_triton-0.1.4-py3-none-any.whl size=28339 sha256=1c0be0adb6a9899223a1150036ced520d534149ecfbef0bd4441e79c885e1204
  Stored in directory: /var/tmp/pip-ephem-wheel-cache-0307bm2h/wheels/fd/a9/d2/a30fdb351289862593a3c1112a76fc9171cad1bedc010aa749
Successfully built ja

In [1]:
from bayes3d.rendering.splat import splat, types, util
import jax
import jax.numpy as jnp
import jax.random as jr
import jax_triton

In [6]:
cam = types.Camera(xyz=jnp.zeros(3), orientation=jnp.eye(3),
                   fx=512., fy=512.,
                   width=1024, height=768)
blobs = jax.vmap(util.generate_random_blob)(jr.split(jr.PRNGKey(0), 1_000_000))
grid = types.Grid(types.Bounds(-4., 4., -3., 3.), nx=cam.width, ny=cam.height)
xys, _ = util.make_image_plane(grid)
im, aux = splat.render(xys, blobs=blobs, camera=cam, grid=grid)
plt.imshow(im, origin='lower')

TritonLoweringException: Exception while lowering eqn:
  a:i32[16,16] = convert_element_type[new_dtype=int32 weak_type=False] b
With context:
  TritonLoweringRuleContext(context=TritonModuleContext(name='_render_img_pallas', ir_context=<triton._C.libtriton.triton.ir.context object at 0x7fe1dc093d30>, builder=<triton._C.libtriton.triton.ir.builder object at 0x7fe1dc0d9950>, module=<triton._C.libtriton.triton.ir.module object at 0x7fe1dc0d9810>, grid_mapping=GridMapping(grid=(48, 64), block_mappings=(BlockMapping(block_shape=(16, 16, 2), index_map_jaxpr={ lambda ; a:i32[] b:i32[]. let  in (a, b, 0) }, memory_space=None), BlockMapping(block_shape=(1, 1), index_map_jaxpr={ lambda ; a:i32[] b:i32[]. let  in (a, b) }, memory_space=None), BlockMapping(block_shape=(1, 1), index_map_jaxpr={ lambda ; a:i32[] b:i32[]. let  in (a, b) }, memory_space=None), BlockMapping(block_shape=(1000000, 3), index_map_jaxpr={ lambda ; a:i32[] b:i32[]. let  in (0, 0) }, memory_space=None), BlockMapping(block_sha

In [2]:
# Aha, we need this JAX commit.
# https://github.com/google/jax/commit/b3904c0508aacb455aa42c25f9bdbb011118b481
import inspect
from jax._src.pallas.triton import lowering
print(inspect.getsource(lowering.lower_jaxpr_to_triton_module))

def lower_jaxpr_to_triton_module(
    jaxpr: jax_core.Jaxpr, in_shapes, grid_mapping: GridMapping, name: str
) -> tl_ir.module:
  jaxpr, _ = pe.dce_jaxpr(jaxpr, [True] * len(jaxpr.outvars), instantiate=True)
  ir_context = tl_ir.context()
  ir_context.load_triton()
  builder = tl_ir.builder(ir_context)
  # TODO(sharadmv): handle multiple devices, right now we assume device 0
  # which is fine when we have multiple of the same GPU but this won't work in
  # general.
  device = 0
  builder.arch = triton_kernel_call_lib.get_compute_capability(device)
  module = builder.create_module()
  in_avals = [var.aval for var in jaxpr.invars]
  triton_types = [get_triton_type(x) for x in in_avals]
  arg_types = [code_gen.str_to_ty(arg) for arg in triton_types]
  assert len(jaxpr.outvars) == 0
  prototype = tl.function_type([], arg_types)
  out = prototype.to_ir(builder)
  fn = builder.get_or_insert_function(module, name, out, "public", False)
  module.push_back(fn)
  entry = fn.add_entry_block()
  a